# Pysam Explained
1. [BAM Files](#BAM)
2. [VCF Files](#VCF)
3. [Fasta Files](#FASTA)



In [30]:
import pysam

## BAM
1. [Opening a BAM File](#Opening)
2. [Using Fetch with BAM](#Fetch)
3. [Pileups and BAM](#Pileup)

### Opening
Opening a BAM File requires you to use the AlignmentFile class. 
AlignmentFile(filepath_or_object, mode=None)
1.  filepath_or_object: Just the filepath/object to the .bam file
2.  mode: r/w for reading/writing sam files; rb/wb for reading/writing bam files

Example: Opening a BAM file for reading. 

In [31]:
bamfile = pysam.AlignmentFile('chr1.sorted.bam', 'rb')

# Returns a tuple of the references in the bamfile
print(bamfile.references)
# Returns a tuple of the legnths of references. 
print(bamfile.lengths)

('chrM', 'chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX', 'chrY')
(16571, 249250621, 243199373, 198022430, 191154276, 180915260, 171115067, 159138663, 146364022, 141213431, 135534747, 135006516, 133851895, 115169878, 107349540, 102531392, 90354753, 81195210, 78077248, 59128983, 63025520, 48129895, 51304566, 155270560, 59373566)


### Fetch
fetch(self, contig=None, start=None, stop=None, region=None, tid=None, until_eof=False, multiple_iterators=False, reference=None, end=None)

fetch returns an iterator of all reads overlapping a region. Sorted by the first algined base. Including reads that are partially overlapping. 

**Fetch only iterates over the reads, if you want to extract information use pileup** 

**Parameters:**

contig: The sequence that a tid refers to. For example chr1, contig123.

tid: The target id. The target id is 0 or a positive integer mapping to entries within the sequence dictionary in the header section of a TAM file or BAM file.

until_eof(bool): If true all reads from the current file position will be returned in order as they are wihtin the file. Using this option will also fetch unmapped reads. 

multiple_iterators(bool): If true, multiple iterators on the same file can be used at the same time. The iterator returned will receive its own copy of a filehandle to the file effectively re-opening the file. Re-opening a file creates some overhead, so beware.

**Return Type:** An iterator over a collection of reads. 

ValueError – if the genomic coordinates are out of range or invalid or the file does not permit random access to genomic coordinates.

Without a contig or region all mapped reads in the file will be fetched. The reads will be returned ordered by reference sequence, which will not necessarily be the order within the file. This mode of iteration still requires an index. If there is no index, use until_eof=True.

If only reference is set, all reads aligned to reference will be fetched.

A SAM file does not allow random access. If region or contig are given, an exception is raised.

In [32]:
# 'Fetches' all reads aligned to region 100000001, look at that mess
for read in bamfile.fetch('chr1', 100000000, 100000001):
    print(read)

HSQ1004:134:C0D8DACXX:2:1107:7396:136241	83	1	99999900	60	101M	1	99999695	101	CTTCATTCGTTTGTTTTGTTGTTTCTATGGCACAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCA	array('B', [18, 34, 34, 34, 33, 31, 33, 27, 23, 32, 23, 32, 27, 33, 32, 27, 32, 30, 33, 30, 25, 25, 34, 31, 34, 34, 34, 32, 25, 26, 20, 12, 34, 34, 34, 34, 34, 34, 36, 34, 32, 32, 30, 30, 30, 33, 33, 30, 37, 39, 39, 33, 37, 31, 36, 39, 38, 38, 36, 40, 40, 38, 37, 36, 34, 38, 31, 36, 40, 37, 38, 39, 36, 40, 40, 39, 39, 33, 39, 40, 40, 40, 36, 37, 36, 31, 37, 37, 27, 32, 30, 37, 37, 36, 35, 30, 30, 37, 31, 34, 31])	[('RG', 'NA12878'), ('XT', 'U'), ('NM', 0), ('SM', 37), ('AM', 37), ('X0', 1), ('X1', 0), ('XM', 0), ('XO', 0), ('XG', 0), ('MD', '101')]
HSQ1004:134:C0D8DACXX:3:2106:6980:17066	147	1	99999903	60	101M	1	99999697	101	CATTCGTTTGTTTTGTTGTTTCTATGGCACAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTG	array('B', [30, 35, 33, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 3

### Pileup
perform a pileup within a region. Alternative way of accessing the data in a SAM/BAM file by iterating over each base of a specified region. Each iteration returns a PileupColumn which represents all the reads in the BAM/SAM file that map to a single base in teh reference sequence. The list of reads are represented as PileupRead objects in the PileupColumn.pileups property. 

Without ‘contig’ or ‘region’ all reads will be used for the pileup. The reads will be returned ordered by contig sequence, which will not necessarily be the order within the file.

Note that SAM formatted files do not allow random access. In these files, if a ‘region’ or ‘contig’ are given an exception is raised.

‘all’ reads which overlap the region are returned. The first base returned will be the first base of the first read ‘not’ necessarily the first base of the region used in the query.

** bamefile.pileup (PileupColumn) -> pileupcol.pileups (PileupRead) -> pileupreads.alignment (AlignedSegment) **

#### PileupColumn
A pileup of reads at a particular reference sequence position (column). A pileup column contains all the reads that map to a certain target base.

**Handy Tools:**
* nsegments: number of reads mapping to this column
* pileups: list of reads (pysam.PileupRead) aligned to this column
* reference_id: the reference sequence number as defined in the header
* reference_name: reference name
* reference_pos: position in the reference sequence (0-based) 

#### PileupRead 
Representation of a read aligned to a particular position in the reference sequence. 

**Handy Tools:** 
* alignment: a pysam.AlignedSegment object of the aligned read
* indel: (insertion or deletion of bases in the genome) indel length for the position following the current pileup site. This quantity peeks ahead to the next cigar operation in this alignment. If the next operation is an insertion, indel will be positive. If the next operation is a deletion, it will be negation. 0 if the next operation is not an indel.
* is_del: 1 iff the base on the padded read is deletion
* is_head: 1 iff the base on the padded read is left-most base. 
* is_refskip: 1 iff the base on the padded read is part of CIGAR N op.
* is_tail: 1 iff the base on the padded read is the right-most base. 
* query_position: position of the read base at the pileup site, 0-based. None if is_del or is_refskip is set.
* query_position_or_next: position of the read base at the pileup site, 0-based. If the current position is a deletion, returns the next aligned base.



#### AlignedSegment
Class representing an aligned segment.

**Useful Information of the Segment**
1. cigarstring: the cigar alignment as a string
2. cigartuples: the cigar alignment.

 Returned as a list of tuples of (operation, length). E.g. [(0, 30)] meaning 30M. 

| Char | Name           | # |
| - | -------------- | - |
| M | BAM_CMATCH     | 0 |
| I | BAM_CINS       | 1 |
| D | BAM_CDEL       | 2 |
| N | BAM_CREF_SKIP  | 3 |
| S | BAM_CSOFT_CLIP | 4 |
| H | BAM_CHARD_CLIP | 5 |
| P | BAM_CPAD       | 6 |
| = | BAM_CEQUAL     | 7 |
| X | BAM_CDIFF      | 8 |
| B | BAM_CBACK      | 9 |


3. compare(self, AlignedSegment other)
 return -1,0,1, if contents in this are binary <,=,> to other

4. mapping_quality
 mapping quality of the specific read to the reference sequence. 
 
5. next_reference_id
 the reference id of the mate/next read. 
 In this case we only grab chr1 so the next will always be 1, the id for chr1. 

6. next_reference_name
 reference name of the mate/next read (None if no AlignmentFile is associated)
 In this case we only grab chr1 so the next will always be chr1. 


**Gets**
1. get_aligned_pairs(self, matches_only=False, with_seq=False)

 a list of aligned read(query) and reference positions
 For inserts, deletions, skipping either query or reference pos may be none
 
 Parameters
 * matches_only(bool): if True, only matched bases are returned - no None on either side
 * with_seq(bool): if True, return a third element in the tuple containing the reference seq. Subs are lower case. This option requires an MD tag to be present. 
 
 Returns: aligned_pairs, list of tuples
2. get_blocks()

 a list of start and end positions of aligned gapless blocks. So in english terms the start and end position for each read with matching

 The start and end positions are in genomic coordinates.

 Blocks are not normalized, i.e. two blocks might be directly adjacent. This happens if the two   blocks are separated by an insertion in the read.
 
3. get_reference_positions(self, full_length=False)

 a list of reference positions that this read aligns to.

 By default, this method only returns positions in the reference that are within the alignment.    If full_length is set, None values will be included for any soft-clipped or unaligned positions    within the read. The returned list will thus be of the same length as the read.
 
4. get_reference_sequence(self)

 return the reference sequence.

 This method requires the MD tag to be set.
 
**IS_THIS**
* is_paired
 true if read is paired in sequencing
* is_reverse
 true if read is mapped to reverse strand
* is_secondary
 true if not primary alignment

**Queries**
* query_alignment_end
 end index of the aligned query portion of the sequence (0-based, exclusive).

 This the index just past the last base in seq that is not soft-clipped.


Examples for looking at CIGAR 

In [33]:
for pileup_col in bamfile.pileup('chr1', 100000000, 100000001):
    for pileup_read in pileup_col.pileups: 
        # Query Position is None if is deletion or is refskip
        if not pileup_read.is_del and not pileup_read.is_refskip:
            print(pileup_read.alignment.get_reference_sequence())
            print(pileup_read.alignment.cigartuples)
            print(pileup_read.alignment.cigarstring)
            

CTTCATTCGTTTGTTTTGTTGTTTCTATGGCACAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCA
[(0, 101)]
101M
CTTCATTCGTTTGTTTTGTTGTTTCTATGGCACAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCA
[(0, 101)]
101M
CTTCATTCGTTTGTTTTGTTGTTTCTATGGCACAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCA
[(0, 101)]
101M
CTTCATTCGTTTGTTTTGTTGTTTCTATGGCACAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCA
[(0, 101)]
101M
CATTCGTTTGTTTTGTTGTTTCTATGGCACAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTG
[(0, 101)]
101M
CTTCATTCGTTTGTTTTGTTGTTTCTATGGCACAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCA
[(0, 101)]
101M
CATTCGTTTGTTTTGTTGTTTCTATGGCACAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTG
[(0, 101)]
101M
CTTCATTCGTTTGTTTTGTTGTTTCTATGGCACAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCA
[(0, 101)]
101M
CATTCGTTTGTTTTGTTGTTTCTATGGCACAGTTATAGTTCCTGGGAGCCCCGCAG

[(0, 101)]
101M
CGTTTGTTTTGTTGTTTCTATGGCACAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAA
[(0, 101)]
101M
CGTTTGTTTTGTTGTTTCTATGGCACAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAA
[(0, 101)]
101M
TTTGTTTTGTTGTTTCTATGGCACAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAAAA
[(0, 101)]
101M
TTGTTTTGTTGTTTCTATGGCACAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAAAAT
[(0, 101)]
101M
TTTGTTGTTTCTATGGCACAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAAAATGGAA
[(0, 101)]
101M
TTTCTATGGCACAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTC
[(0, 101)]
101M
TTCTATGGCACAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCC
[(0, 101)]
101M
CTATGGCACAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCA
[(0, 101)]
101M
GCACAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGT

101M
TTCTATGGCACAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCC
[(0, 101)]
101M
CTATGGCACAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCA
[(0, 101)]
101M
GCACAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTaTTCTGaCACTATATATCTAGCACtTGCACTgTAAAAaTGGAAGTAATTCCCATT
[(0, 98), (4, 3)]
98M3S
CAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACC
[(0, 101)]
101M
CAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACC
[(0, 101)]
101M
GTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAG
[(0, 101)]
101M
GTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAG
[(0, 101)]
101M
ATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAA
[(0, 101)]
101M
TAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTA

101M
CATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAG
[(0, 100), (4, 1)]
100M1S
CATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGT
[(0, 101)]
101M
TGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAA
[(0, 101)]
101M
GTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAAAATGgAAGTAATTCCCATTAgGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAA
[(0, 101)]
101M
CTTCATTCGTTTGTTTTGTTGTTTCTATGGCACAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCA
[(0, 101)]
101M
CATTCGTTTGTTTTGTTGTTTCTATGGCACAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTG
[(0, 101)]
101M
TCGTTTGTTTTGTTGTTTCTATGGCACAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAA
[(0, 101)]
101M
CGTTTGTTTTGTTGTTTCTATGGCACAGTTATAGTTCCTGGGAGCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAA
[(0, 101)]
101M
CGTTTGTTTTGTTGTTTCTATGGCACAGTTATAGTTCCTGGG

GCCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAA
[(0, 101)]
101M
CCCCGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAA
[(0, 101)]
101M
CGCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAA
[(0, 101)]
101M
GcAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAA
[(0, 101)]
101M
GCAGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAA
[(0, 101)]
101M
AGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGA
[(0, 101)]
101M
AGAACATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGA
[(0, 101)]
101M
CATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAG
[(0, 100), (4, 1)]
100M1S
CATGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGTAAA

...

[(0, 101)]
101M
TATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTG
[(0, 101)]
101M
TATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTGAATT
[(0, 101)]
101M
TATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTGAATT
[(0, 101)]
101M
TCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTGAATTTT
[(0, 101)]
101M
CTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTGAATTTTA
[(0, 101)]
101M
CTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTGAATTTTA
[(0, 101)]
101M
CTAGCAcTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACaGTAAAAGCTCATGCCAAAAGCTGAATTTTA
[(0, 101)]
101M
TAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTGAATTTTAC
[(0, 101)]
101M
TAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGG

[(0, 101)]
101M
TAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTGAATTTTA
[(0, 100), (4, 1)]
100M1S
TAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTGAATTTTAC
[(0, 101)]
101M
AGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTGAATTTTACT
[(0, 101)]
101M
GCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTGAATTTTACTT
[(0, 101)]
101M
GCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTGAATTTTACTT
[(0, 101)]
101M
CTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTGAATTTTACTTAAT
[(0, 101)]
101M
TGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTGAATTTTACTTAATAT
[(0, 101)]
101M
GCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTGAATTTTACTTAATATA
[(0, 101)]
101M
CTGGGAGCCCCGCAGAACATGGTGTTTTATT

101M
CTGACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCA
[(0, 101)]
101M
CTGACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCA
[(0, 101)]
101M
CTGACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCA
[(0, 101)]
101M
TGACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAA
[(0, 101)]
101M
GACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAA
[(0, 101)]
101M
ACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGC
[(0, 101)]
101M
CTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCT
[(0, 101)]
101M
TATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTG
[(0, 101)]
101M
TATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCA

101M
TAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTGAATTTTA
[(0, 100), (4, 1)]
100M1S
TAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTGAATTTTAC
[(0, 101)]
101M
AGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTGAATTTTACT
[(0, 101)]
101M
GCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTGAATTTTACTT
[(0, 101)]
101M
GCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTGAATTTTACTT
[(0, 101)]
101M
CTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTGAATTTTACTTAAT
[(0, 101)]
101M
TGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTGAATTTTACTTAATAT
[(0, 101)]
101M
GCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTGAATTTTACTTAATATA
[(0, 101)]
101M
TGGTGTTTTATTCTGACACTATATATCTAGCACTTGCACTGT

TGACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAA
[(0, 101)]
101M
GACACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAA
[(0, 101)]
101M
ACTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGC
[(0, 101)]
101M
CTATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCT
[(0, 101)]
101M
TATATATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTG
[(0, 101)]
101M
TATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTGAATT
[(0, 101)]
101M
TATCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTGAATT
[(0, 101)]
101M
TCTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGAGGCTAAAAAAAGACAGTAAAAGCTCATGCCAAAAGCTGAATTTT
[(0, 101)]
101M
CTAGCACTTGCACTGTAAAAATGGAAGTAATTCCCATTAGGACCAGCAAAACCTGA

Examples of query functions
**query_alignment_length is not equivalent to query_length**
query_length is the length of the whole query whereas the prior is the length of algined sequence. 
**get_reference_sequence is identical to query_alignment_sequence**


In [34]:
for pileup_col in bamfile.pileup('chr1', 100000000, 100000001):
    for pileup_read in pileup_col.pileups: 
        # Query Position is None if is deletion or is refskip
        if not pileup_read.is_del and not pileup_read.is_refskip:
            print(pileup_read.alignment.query_alignment_sequence)

...


...


...


...


...


...

...


...


...


...


...


In [35]:
# Always remember to close your bamfile!
bamfile.close()

## VCF
1. Opening a VCF File 

### Opening a VCF File
Opening a VCF File requires you to use the VariantFile class. VariantFile(filename, mode=None, drop_samples=False)
1. filename: Just the filepath/object to the .vcf file 
2. mode: r/w for reading/writing vcf files; rb/wb for reading/writing bcf files
3. drop_samples(bool): Ignore sample information when reading. 

Example: Opening a vcf file for reading. Note if mode not dectected it will guess. For most files this will work like in the one below. 

In [36]:
vcf_in = pysam.VariantFile('NA12878_S1.genome.vcf.gz')

### Fetch
fetch(self, contig=None, start=None, stop=None, region=None, reopen=False, end=None, reference=None)

fetch records in a region (uses 0-based indexing). The region is specified by **contig, start and end**. Without those all mapped records will be fetched. Order will be by config not necessarily by order within the file. 

**reopen** will be true if you will be using multiple iterators on the same file at the same time. 


VariantRecord
**Useful Functions**
* alleles
 tuple of reference allele followed by alt alleles
* contig
 chromosome/contig name
* copy()
 returns a copy of the Variant Record object
* ref
 reference allele
* rlen
 record length on chrom/contig 
* start/stop
 record start and stop position on chrom/contig (0-based)
* qual
 phred scaled quality score or None if not available

In [37]:
for rec in vcf_in.fetch('chr1', 100000, 100001):
    print('Chromosome:', rec.contig)
    print('Position:', rec.pos)
    print('Allele:', rec.alleles)
    print('Reference Allele:', rec.ref)
    print('Length:', rec.rlen)
    print('Start Pos:', rec.start)
    print('Stop Pos:', rec.stop)
    

Chromosome: chr1
Position: 99796
Allele: ('C',)
Reference Allele: C
Length: 853
Start Pos: 99795
Stop Pos: 100648


In [38]:
# Always remember to close your file!
vcf_in.close()

## FASTA
1. [Opening a FASTA File](#Opening)
2. [Using fetch with FASTA](#Fetch)
3. [Reference Length](#Reference_Length)
4. [Useful Functions](#Useful_Functions)

### Opening
Opening a FASTA file requires you to use the FastaFile class. FastaFile(filename)
1. filename: Just the filepath/object to the fasta file. 

Raises:	

ValueError – if index file is missing

IOError – if file could not be opened

In [39]:
fasta_in = pysam.FastaFile('chr1.fa')

### Fetch
fetch(self, reference=None, start=None, end=None, region=None)

Fetch sequences in a region.

A region can either be specified by reference, start and end. 

Alternatively, a samtools region string can be supplied.

If any of the coordinates are missing they will be replaced by the minimum (start) or maximum (end) coordinate.

Note that region strings are 1-based, while start and end denote an interval in python coordinates. The region is specified by reference, start and end.

Returns: string

Return type: a string with the sequence specified by the region.

Raises:	
IndexError – if the coordinates are out of range
ValueError – if the region is invalid

In [40]:
# Returns a string with the sequence 
sequence = fasta_in.fetch('chr1', start=100000, end=100005)
print(sequence)
# Iterates over each character in the string 
for x in fasta_in.fetch('chr1', start=100000, end=100005):
    print(x)

actaa
a
c
t
a
a


### Reference_Length
get_reference_length(self, reference)

Return the length of reference in a form of an int.

In [41]:
length_of_reference = fasta_in.get_reference_length('chr1')
print(length_of_reference)

249250621


### Useful_Functions
1. is_open(self): Returns True or False based on if a FASTA file has been open or not
2. lengths: Returns a tuple of lengths of references in the specific FASTA file
3. nreferences: Returns an int of the number of references a FASTA File has
4. references: Returns a tuple of the references in the FASTA file

In [42]:
print('Is Open?:', fasta_in.is_open())
print('Length:', fasta_in.lengths)
print('Number of References:', fasta_in.nreferences)
print('References:', fasta_in.references)

Is Open?: True
Length: [249250621]
Number of References: 1
References: ['chr1']


In [43]:
fasta_in.close()